In [6]:
import json
import csv

In [4]:
with open('data/raw/worldbank-emissions.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        with open('data/emissions/' + row['Country Name'] + '.json', 'w') as f:
            json.dump({
                'x': list(range(1960, 2018)),
                'y': [row[str(i)] for i in range(1960, 2018)],
                'mode': 'lines+markers',
                'name': row['Country Name'] + "(Total CO2 Emissions in metric tons per capita)",
            }, f)

In [6]:
with open('data/raw/worldbank-emissions_total.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        with open('data/emissions_total/' + row['Country Name'] + '.json', 'w') as f:
            json.dump({
                'x': list(range(1960, 2018)),
                'y': [row[str(i)] for i in range(1960, 2018)],
                'mode': 'lines+markers',
                'name': row['Country Name'] + "(CO2 emissions in kt)",
            }, f)

In [19]:
data = {'labels': [], 'parents': [], 'values': []}
categories = ['Transport (tonnes)', 'Energy (tonnes)', 'Other sources (tonnes)', 'Agriculture, Land Use & Forestry (tonnes)', 'Waste (tonnes)', 'Residential & commercial (tonnes)', 'Industry (tonnes)', 'Agriculture (tonnes)']

with open('data/raw/ourworldindata-global-carbon-dioxide-emissions-by-sector.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Year'] != '2010':
            continue

        if row['Entity'] == 'World':
            data['labels'].append(row['Entity'])
            data['parents'].append('')
            data['values'].append(sum(float(row[c]) for c in categories))
            
            for c in categories:
                data['labels'].append(c[:-9])
                data['parents'].append('World')
                data['values'].append(row[c])
            
            continue
        
        if not row['Code']:
            continue

        for c in categories:
            # Forestry contains negative values, we need to cut at 200M so the sum isn't bigger than the total
            # otherwise plotly can't render :)
            if row[c] and float(row[c]) > 50000000 and ('Forestry' not in c or float(row[c]) > 200000000):
                data['labels'].append(row['Entity'])
                data['parents'].append(c[:-9])
                data['values'].append(row[c])
            
with open('data/sector_sunburst.json', 'w') as f:
    json.dump({
        'type': 'sunburst',
        'maxdepth': 2,
        'labels': data['labels'],
        'parents': data['parents'],
        'values': [float(v)/1000000 for v in data['values']],
        'name': 'CO2 Emissions per Sector',
        'branchvalues': 'total',
    }, f)